In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
import os
import zipfile
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy, Precision, Recall, AUC
from tensorflow.keras.optimizers import Adam

In [70]:
# Define dataset directories and image size
input_dir = '/content/drive/MyDrive/CRUDAS'  # Directory containing input images
output_dir = '/content/drive/MyDrive/LIMPIAS'  # Directory containing segmented images
image_size = 100  # Adjust according to your requirements

In [71]:
# Function to load and preprocess data from a directory
def load_data(input_dir, output_dir, image_size):
    input_images = []
    output_images = []

    for filename in os.listdir(input_dir):
        if filename.endswith(".jpeg") or filename.endswith(".png"):
            input_image_path = os.path.join(input_dir, filename)
            output_image_path = os.path.join(output_dir, filename)

            # Load and preprocess input image
            input_image = cv2.imread(input_image_path)
            input_image = cv2.resize(input_image, image_size)
            input_image = input_image / 255.0

            # Load and preprocess output image
            output_image = cv2.imread(output_image_path, cv2.IMREAD_GRAYSCALE)
            output_image = cv2.resize(output_image, image_size)
            output_image = np.expand_dims(output_image, axis=-1) / 255.0

            input_images.append(input_image)
            output_images.append(output_image)

    return np.array(input_images), np.array(output_images)


In [72]:
# Function to create U-Net model
def create_unet_model(input_shape):
    model = Sequential([
        # Encoder
        Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        # Decoder
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        UpSampling2D((2, 2)),
        Conv2D(1, (3, 3), activation='sigmoid', padding='same')
    ])
    return model

In [73]:
# Load and preprocess data
X, y = load_data(input_dir, output_dir, image_size)


In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

Shape of X_train: (6, 126, 126, 3)
Shape of y_train: (6, 126, 126, 1)


In [44]:
print(f"First 5 elements of X: {X[:5]}")
print(f"First 5 elements of y: {y[:5]}")

First 5 elements of X: []
First 5 elements of y: []


In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
# Create and compile the U-Net model
model = create_unet_model(input_shape=(image_size[0], image_size[1], 3))
model.compile(optimizer=Adam(), loss=BinaryCrossentropy(), metrics=[Accuracy(), Precision(), Recall(), AUC()])

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
# Evaluate model performance
train_metrics = model.evaluate(X_train, y_train)
test_metrics = model.evaluate(X_test, y_test)

In [ ]:
print("Training Metrics:")
print("Loss:", train_metrics[0])
print("Accuracy:", train_metrics[1])
print("Precision:", train_metrics[2])
print("Recall:", train_metrics[3])
print("AUC:", train_metrics[4])


In [ ]:
print("\nTesting Metrics:")
print("Loss:", test_metrics[0])
print("Accuracy:", test_metrics[1])
print("Precision:", test_metrics[2])
print("Recall:", test_metrics[3])
print("AUC:", test_metrics[4])

In [ ]:
# Perform segmentation on test images
segmented_images = model.predict(X_test)

# Display test images and segmented images for visual inspection
for i in range(len(X_test)):
    cv2.imshow('Test Image', X_test[i])
    cv2.imshow('Segmented Image', segmented_images[i])
    cv2.waitKey(0)

cv2.destroyAllWindows()